In [1]:
!rm cifar-10-python*
!wget "http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
!ls

rm: cannot remove 'cifar-10-python*': No such file or directory
--2021-12-13 22:54:33--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  96.9MB/s    in 1.7s    

2021-12-13 22:54:35 (96.9 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-python.tar.gz	sample_data


In [2]:
!tar -xvzf cifar-10-python.tar.gz

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [3]:
import os
import pickle
import numpy as np
from pathlib import Path

In [4]:
!ls

cifar-10-batches-py  cifar-10-python.tar.gz  sample_data


In [5]:
def shuffle_data(data_x, data_y):
  indices = np.arange(data_x.shape[0])
  np.random.shuffle(indices)
  shuffled_data_x = np.ascontiguousarray(data_x[indices])
  shuffled_data_y = np.ascontiguousarray(data_y[indices])
  return shuffled_data_x, shuffled_data_y

def unpickle(file):
  fo = open(file, 'rb')
  dict = pickle.load(fo, encoding='latin1')
  fo.close()
  return dict

DATA_DIR = 'cifar-10-batches-py/'

img_height = 32
img_width = 32
num_channels = 3
num_classes = 10

train_x = np.ndarray((0, img_height * img_width * num_channels), dtype=np.float32)
train_y = []
for i in range(1, 6):
  subset = unpickle(os.path.join(DATA_DIR, 'data_batch_%d' % i))
  train_x = np.vstack((train_x, subset['data']))
  train_y += subset['labels']
train_x = train_x.reshape((-1, num_channels, img_height, img_width)).transpose(0, 2, 3, 1)
train_y = np.array(train_y, dtype=np.int32)

subset = unpickle(os.path.join(DATA_DIR, 'test_batch'))
test_x = subset['data'].reshape((-1, num_channels, img_height, img_width)).transpose(0, 2, 3, 1).astype(np.float32)
test_y = np.array(subset['labels'], dtype=np.int32)

valid_size = 5000
train_x, train_y = shuffle_data(train_x, train_y)
valid_x = train_x[:valid_size, ...]
valid_y = train_y[:valid_size, ...]
train_x = train_x[valid_size:, ...]
train_y = train_y[valid_size:, ...]
data_mean = train_x.mean((0, 1, 2))
data_std = train_x.std((0, 1, 2))

train_x = (train_x - data_mean) / data_std
valid_x = (valid_x - data_mean) / data_std
test_x = (test_x - data_mean) / data_std

train_x = train_x.transpose(0, 3, 1, 2)
valid_x = valid_x.transpose(0, 3, 1, 2)
test_x = test_x.transpose(0, 3, 1, 2)


Dio dolje iskopiran iz 1. zadatka (Luka)




In [6]:
from numpy import floor
from torch import nn

class ConvolutionalModel(nn.Module):

    def __init__(self, in_channels, in_width, conv1_channels, pool1_width, conv2_channels, pool2_width, fc3_width, fc4_width, class_count):
        super(ConvolutionalModel, self).__init__()
                                                                                                                # in_channels x in_width x in_width
        self.conv1 = nn.Conv2d(in_channels, conv1_channels, kernel_size=5, stride=1, padding=2, bias=True)      # conv1_channels x in_width x in_width
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(pool1_width, stride=2)                                                        # conv1_channels x w2 x w2

        w2 = floor((in_width - pool1_width) / 2 + 1)

        self.conv2 = nn.Conv2d(conv1_channels, conv2_channels, kernel_size=5, stride=1, padding=2, bias=True)   # conv2_channels x w2 x w2
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(pool2_width, stride=2)                                                        # conv2_channels x w3 x w3
        
        w3 = floor((w2 - pool2_width) / 2 + 1)

        self.flatten3 = nn.Flatten()                                                                            # (conv2_channels x w3 x w3)
        self.fc3 = nn.Linear((int)(conv2_channels * w3 * w3), fc3_width)                                        # fc3width
        self.relu3 = nn.ReLU()

        self.fc4 = nn.Linear(fc3_width, fc4_width)                                                              # fc4width
        self.relu4 = nn.ReLU()                                                       

        self.fc_logits = nn.Linear(fc4_width, class_count)                                                      # class_count

        self.reset_parameters()

    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear) and m is not self.fc_logits:
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)
        self.fc_logits.reset_parameters()

    def forward(self, x):
        h = self.conv1(x)
        h = self.relu1(h)
        h = self.pool1(h)

        h = self.conv2(h)
        h = self.relu2(h)
        h = self.pool2(h)
        
        h = self.flatten3(h)
        h = self.fc3(h)
        h = self.relu3(h)

        h = self.fc4(h)
        h = self.relu4(h)

        logits = self.fc_logits(h)
        return logits

In [7]:
import torch

data_channels = 3
data_width = 32
CifarCnn = ConvolutionalModel(data_channels, data_width, 16, 3, 32, 3, 256, 128, 10)

inp = torch.randn(1, 3, 32, 32)
out = CifarCnn(inp)
print(out)

tensor([[-0.6008,  0.1036, -1.0019, -0.0990,  0.4445, -0.7669, -0.8683,  0.6521,
         -2.4161,  2.4411]], grad_fn=<AddmmBackward0>)


In [8]:
valid_values = []

for i in valid_y:
  appended = np.zeros(10)
  appended[i] = 1
  valid_values.append(appended)

valid_labels = np.array(valid_values)

train_values = []

for i in train_y:
  appended = np.zeros(10)
  appended[i] = 1
  train_values.append(appended)
train_labels = np.array(train_values)



In [9]:
print(valid_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [10]:
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

In [11]:
def evaluate(model, criterion, x_in, y_in):
  # np array u tensor
  x = torch.FloatTensor(x_in)
  y = torch.FloatTensor(y_in)

  outputs = model.forward(x)
  loss = criterion(outputs, y)

  total = len(y)
  temp_list = []
  for test in y:
    temp_list.append(np.where(test==1)[0][0])

  targets_np = np.array(temp_list)
  _, predicted_tensor = outputs.max(1)
  predicted_np = predicted_tensor.detach().numpy()
  correct = np.sum(targets_np==predicted_np)
  accuracy = correct / total

  return float(loss.detach().numpy()), accuracy

In [12]:
plot_data = {}
plot_data['train_loss'] = []
plot_data['valid_loss'] = []
plot_data['train_acc'] = []
plot_data['valid_acc'] = []
plot_data['lr'] = []

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(CifarCnn.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
num_epochs = 5
n_batch=500 #bsz izračunato iz duljine trening seta i broja batcheva
bsz = len(train_x) // n_batch
for epoch in range(num_epochs):
    X, Yoh = shuffle_data(train_x, train_labels)
    X = torch.FloatTensor(X)
    Yoh = torch.FloatTensor(Yoh)
    for batch in range(n_batch):
        # broj primjera djeljiv s veličinom grupe bsz
        batch_X = X[batch*bsz:(batch+1)*bsz, :]
        batch_Yoh = Yoh[batch*bsz:(batch+1)*bsz, :]
        outputs = CifarCnn.forward(batch_X)

        loss = criterion(outputs, batch_Yoh)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch%50 == 0:
            print("epoch: {}, step: {}/{}, batch_loss: {}".format(epoch, batch, n_batch, loss))

    valid_loss, valid_accuracy = evaluate(CifarCnn, criterion, valid_x, valid_labels)
    print(f"valid_loss:{valid_loss} , valid_accuracy:{valid_accuracy}")
    
    lr_scheduler.step()

epoch: 0, step: 0/500, batch_loss: 2.599403142929077
epoch: 0, step: 50/500, batch_loss: 1.773411750793457
epoch: 0, step: 100/500, batch_loss: 1.5525628328323364
epoch: 0, step: 150/500, batch_loss: 1.6426035165786743
epoch: 0, step: 200/500, batch_loss: 1.4936728477478027
epoch: 0, step: 250/500, batch_loss: 1.3914496898651123
epoch: 0, step: 300/500, batch_loss: 1.3761553764343262
epoch: 0, step: 350/500, batch_loss: 1.2882521152496338
epoch: 0, step: 400/500, batch_loss: 1.15902578830719
epoch: 0, step: 450/500, batch_loss: 1.1230683326721191
valid_loss:1.1793975830078125 , valid_accuracy:0.5772
epoch: 1, step: 0/500, batch_loss: 1.1701619625091553
epoch: 1, step: 50/500, batch_loss: 1.1129745244979858
epoch: 1, step: 100/500, batch_loss: 0.8968616127967834
epoch: 1, step: 150/500, batch_loss: 1.032361626625061
epoch: 1, step: 200/500, batch_loss: 1.1367759704589844
epoch: 1, step: 250/500, batch_loss: 0.9920256733894348
epoch: 1, step: 300/500, batch_loss: 0.9115530848503113
epoch